<a href="https://colab.research.google.com/github/vishnubanna/learningmachinelearning/blob/master/yolo3net_IN_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install tensorflow==2.0.0-rc0

     |████████████████████████████████| 86.3MB 1.2MB/s 
     |████████████████████████████████| 501kB 27.7MB/s 
     |████████████████████████████████| 4.3MB 31.3MB/s 
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as ks
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *

#class DetectionLayer(ks.layers.Layers):

class YoloNet(ks.Model):
    @staticmethod
    def skipblock(inp, out):
        x = Add()([inp,out])
        x = ks.activations.linear(x)
        return x
    
    # yolo does not do downsampling via maxpooling, it only uses convolution to make filters and downsample
    # this is done to prevent the degredation of the images detail as the image proceeds int he network 
    # max pooling blurs image, so as the layers gather up the smaller details in the image are lost
    # with an object detecter you are trying to find multiple objects in one image. as a reult using max pooling to down scale the image could 
    # result in the loss of small details ORRRRRR the loss of an object that is in the image and very small 
    # 
    # figure out why they use convolutions of stride (1,1) and kernel size (1,1)., 
        #i think this is being done to look at each individual segment in the image so the detector can look at each chunk and detect objects
        #if shit falls through then try skipping between stride (1,1) and kernel size (1,1) layers, this will allow predictions decision to skip past or between detection blocks block is some cases
    @staticmethod
    def blockmain(xin1, filternum, pad, skip = True):
        xin = Conv2D(filters = filternum, kernel_size = (3,3), strides = (2,2), padding = pad)(xin1)
        #understandably the stride of (2,2) down samples the image while the stride of (1,1) does not
        xin = LeakyReLU(alpha = 0.3)(xin)
        x = Conv2D(filters = int(filternum/2), kernel_size = (1,1), strides = (1,1), padding = pad)(xin)
        x = LeakyReLU(alpha = 0.3)(x)
        x = Conv2D(filters = filternum, kernel_size = (3,3), strides = (1,1), padding = pad)(x)
        #in terms of downsampling (1,1) with "valid", it reuces the by 2 colomns and 2 rows, 13 --> 11, but with "same" padding, the matrix is only down samplif if the stride is > (1,1)
        #in any other case the matrix is padded with zeros 
        x = LeakyReLU(alpha = 0.3)(x)
        if skip == True: 
            x = YoloNet.skipblock(x, xin)
        return x

    @staticmethod
    def subblock(xin, filternum, pad, skip = True):
        x = Conv2D(filters = int(filternum/2), kernel_size = (1,1), strides = (1,1), padding = pad)(xin)
        x = LeakyReLU(alpha = 0.3)(x)
        x = Conv2D(filters = filternum, kernel_size = (3,3), strides = (1,1), padding = pad)(x)
        x = LeakyReLU(alpha = 0.3)(x)
        if skip == True: 
            x = YoloNet.skipblock(x, xin)
        return x
    
    def subblocknoskip(xin, filternum, pad):
        x = Conv2D(filters = int(filternum/2), kernel_size = (1,1), strides = (1,1), padding = pad)(xin)
        x = LeakyReLU(alpha = 0.3)(x)
        x = Conv2D(filters = filternum, kernel_size = (3,3), strides = (1,1), padding = pad)(x)
        x = LeakyReLU(alpha = 0.3)(x)
        return x

    @staticmethod
    def build(h, w, s):
        inputShape = (h,w,3)
        pad = "same"
        outsize = 7

        inp = Input(shape = inputShape, name = 'img')
        #inp = tf.dtypes.cast(inp, dtype = tf.float32)
        bias = Conv2D(filters = 32, kernel_size = (3,3), strides = (1,1), padding  = pad)(inp)
        connect = YoloNet.blockmain(bias, filternum = 64, pad = "same", skip = False)
        connect = YoloNet.blockmain(connect, filternum = 128, pad = "same") #, skip = False)
        connect = YoloNet.subblock(connect, filternum = 128, pad = "same")
        connect = YoloNet.blockmain(connect, filternum = 256, pad = "same")#, skip = False)
        connect = YoloNet.subblock(connect, filternum = 256, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 256, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 256, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 256, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 256, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 256, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 256, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 256, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 256, pad = "same")
        connect = YoloNet.blockmain(connect, filternum = 512, pad = "same")#, skip = False)
        connect = YoloNet.subblock(connect, filternum = 512, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 512, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 512, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 512, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 512, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 512, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 512, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 512, pad = "same")
        connect = YoloNet.blockmain(connect, filternum = 1024, pad = "same")#, skip = False)
        connect = YoloNet.subblock(connect, filternum = 1024, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 1024, pad = "same")
        connect = YoloNet.subblock(connect, filternum = 1024, pad = "same")
        #
        connect = YoloNet.subblocknoskip(connect, filternum = 1024, pad = "same")
        connect = YoloNet.subblocknoskip(connect, filternum = 1024, pad = "same")
        connect = YoloNet.subblocknoskip(connect, filternum = 1024, pad = "same")
        # bt = YoloNet.subblock(bs, filternum = 1024, pad = "same")
        # bu = YoloNet.subblock(bt, filternum = 1024, pad = "same")
        final = Conv2D(filters = 255, kernel_size = (3,3), strides = (1,1), padding = pad)(connect)
        connect = Conv2D(filters = 128, kernel_size = (1,1), strides = (1,1), padding = pad)(final)
        connect = Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), padding = pad)(connect)
        connect = Conv2D(filters = 128, kernel_size = (1,1), strides = (1,1), padding = pad)(connect)
        connect = Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), padding = pad)(connect)
        connect = Conv2D(filters = 128, kernel_size = (1,1), strides = (1,1), padding = pad)(connect)
        connect = Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), padding = pad)(connect)
        connect = Conv2D(filters = outsize, kernel_size = (1,1), strides = (1,1), padding = pad)(connect)
        final1 = UpSampling2D(size = (2,2), interpolation = 'bilinear')(connect)
        connect = Conv2D(filters = 128, kernel_size = (1,1), strides = (1,1), padding = pad)(final1)
        connect = Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), padding = pad)(connect)
        connect = Conv2D(filters = 128, kernel_size = (1,1), strides = (1,1), padding = pad)(connect)
        connect = Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), padding = pad)(connect)
        connect = Conv2D(filters = 128, kernel_size = (1,1), strides = (1,1), padding = pad)(connect)
        connect = Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), padding = pad)(connect)
        connect = Conv2D(filters = outsize, kernel_size = (1,1), strides = (1,1), padding = pad)(connect)
        final2 = UpSampling2D(size = (2,2), interpolation = 'bilinear')(connect)
        connect = Conv2D(filters = 128, kernel_size = (1,1), strides = (1,1), padding = pad)(final1)
        connect = Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), padding = pad)(connect)
        connect = Conv2D(filters = 128, kernel_size = (1,1), strides = (1,1), padding = pad)(connect)
        connect = Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), padding = pad)(connect)
        connect = Conv2D(filters = 128, kernel_size = (1,1), strides = (1,1), padding = pad)(connect)
        connect = Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), padding = pad)(connect)
        final3 = Conv2D(filters = outsize, kernel_size = (1,1), strides = (1,1), padding = pad)(connect)
        
        model = ks.Model(inputs = [inp],
                         outputs = [final1, final2, final3],
                         name = 'YoloNet')
        return model

model = YoloNet.build(416,416,3)
model.summary()


Model: "YoloNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 416, 416, 3) 0                                            
__________________________________________________________________________________________________
conv2d_716 (Conv2D)             (None, 416, 416, 32) 896         img[0][0]                        
__________________________________________________________________________________________________
conv2d_717 (Conv2D)             (None, 208, 208, 64) 18496       conv2d_716[0][0]                 
__________________________________________________________________________________________________
leaky_re_lu_630 (LeakyReLU)     (None, 208, 208, 64) 0           conv2d_717[0][0]                 
____________________________________________________________________________________________